<img align=right src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=150>
<img src=images/acme.jpeg width=100 align=left>

# Vault Setup (dev mode)

**"Dev"** mode is a pre-configured server
* useful for local development, testing, and exploration
* not very secure
* in-memory
* Vault is automatically unsealed
* can optionally set the initial root token


Prereq: 
You need to have Vault binary


Command to execute (in separate windows):

`VAULT_UI=true VAULT_REDIRECT_ADDR=http://127.0.0.1:8200 vault server -log-level=trace -dev -dev-root-token-id=root -dev-listen-address=127.0.0.1:8200 -dev-ha -dev-transactional`


In [2]:
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_SKIP_VERIFY=true

In [2]:
vault --version

Vault v1.6.0 (7ce0bd9691998e0443bc77e98b1e2a4ab1e965d4)


In [9]:
unset VAULT_TOKEN
vault login root

Success! You are now authenticated. The token information displayed below
is already stored in the token helper. You do NOT need to run "vault login"
again. Future Vault requests will automatically use this token.

Key                  Value
---                  -----
token                root
token_accessor       DX9mkWrAfEN0wRJaMBCuy6hN
token_duration       ∞
token_renewable      false
token_policies       ["root"]
identity_policies    []
policies             ["root"]


#### Note the Initialized and Sealed status above.

## Interacting with Vault

Vault **HTTP API**
* Full access to Vault via HTTP
* Every aspect of Vault can be controlled via API


Vault **CLI**
* uses HTTP API to access Vault
* it is a thin wrapper around the HTTP API
* outputs are formatted

**Note**: You should remember **-output-curl-string** CLI flag to find out the `curl` equivalent of CLI.

## Enable Audit Logging

Audit backend keeps a detailed log of all requests and responses to Vault.  Sensitive information is **obfuscated** by default (HMAC).

In [6]:
vault audit enable file file_path=/tmp/audit.log

Success! Enabled the file audit device at: file/


In [3]:
cat /tmp/audit.log |jq

{
  "time": "2020-12-24T07:24:23.030504Z",
  "type": "response",
  "auth": {
    "client_token": "hmac-sha256:43510274735afea704524fe636af99dfdd216b0ade274ae07eeb1044c6ccce32",
    "accessor": "hmac-sha256:29a3538aa4c60394043cb9065977e6b7095420f6986aede129ebd2cc223ef5de",
    "display_name": "token",
    "policies": [
      "root"
    ],
    "token_policies": [
      "root"
    ],
    "token_type": "service",
    "token_issue_time": "2020-12-24T15:23:11+08:00"
  },
  "request": {
    "id": "31d40fbb-9be4-99ab-eeba-519a539b6360",
    "operation": "update",
    "mount_type": "system",
    "client_token": "hmac-sha256:43510274735afea704524fe636af99dfdd216b0ade274ae07eeb1044c6ccce32",
    "client_token_accessor": "hmac-sha256:29a3538aa4c60394043cb9065977e6b7095420f6986aede129ebd2cc223ef5de",
    "namespace": {
      "id": "root"
    },
    "path": "sys/audit/file",
    "data": {
      "description": "hmac-sha256:37d76557af31a8b85a44e9f578593527d34a6a31e68541daa9fe3107362bc3d3",
      "loca

In [8]:
vault audit enable -path=file-raw file file_path=/tmp/audit-raw.log log_raw=true

Success! Enabled the file audit device at: file-raw/


In [6]:
cat /tmp/audit-raw.log |jq

{
  "time": "2020-12-24T07:26:33.060622Z",
  "type": "response",
  "auth": {
    "client_token": "root",
    "accessor": "fJ8e3QTuMKunEyab9ODjEA1M",
    "display_name": "token",
    "policies": [
      "root"
    ],
    "token_policies": [
      "root"
    ],
    "token_type": "service",
    "token_issue_time": "2020-12-24T15:23:11+08:00"
  },
  "request": {
    "id": "3cd1cf10-9fde-2390-897a-59a1fccc76c8",
    "operation": "update",
    "mount_type": "system",
    "client_token": "root",
    "client_token_accessor": "fJ8e3QTuMKunEyab9ODjEA1M",
    "namespace": {
      "id": "root"
    },
    "path": "sys/audit/file-raw",
    "data": {
      "description": "",
      "local": false,
      "options": {
        "file_path": "/tmp/audit-raw.log",
        "log_raw": "true"
      },
      "type": "file"
    },
    "remote_address": "127.0.0.1"
  },
  "response": {
    "mount_type": "system"
  }
}


---
## Generate a root token

Good Practice: The root tokens should be used only for just enough initial setup or in emergencies.  Use tokens with assigned set of policies based on your role in the organization.

When a situation arise and you need a root token, this task walks you through the steps to generate one

#### 1a. Initialize the root generation with One Time Password (OTP)

In [3]:
vault operator generate-root -generate-otp > /tmp/otp.txt

In [4]:
cat /tmp/otp.txt

AwJA1xqvzR4Vdu8996NYMmwWS4

&nbsp;   
#### 1b. Generate nonce to be distributed to all unseal key holders

In [5]:
vault operator generate-root -init -otp=$(cat /tmp/otp.txt) -format=json | jq -r ".nonce" > /tmp/nonce.txt

&nbsp;   
#### 2. Each key holder run 'generate-root' with their unseal key
Note:  The generation of a root token requires a quorum of unseal keys.

In [13]:
vault operator generate-root -nonce=$(cat /tmp/nonce.txt) "anvKNuOKnLkTd5p4dQDfZ5Bv2b4We4tEIJXJ+divvVY="

Nonce            0a1d941a-fb30-7bc6-1057-f8b3f9fee295
Started          true
Progress         1/1
Complete         true
Encoded Token    FF8+HgUWAgtdPSQTYiAsUDksEFYKLRkSBks


#### 3. Decode the generated root token
The previous output should have **Encoded Token** which then need to be decoded

In [14]:
vault operator generate-root -decode="FF8+HgUWAgtdPSQTYiAsUDksEFYKLRkSBks" -otp=$(cat /tmp/otp.txt)

s.SxNruZ2GrTRBi3qgg29gTPRx


In [16]:
vault token lookup s.SxNruZ2GrTRBi3qgg29gTPRx

Key                 Value
---                 -----
accessor            QBPktcYWq9FvkRcqkij6FLd7
creation_time       1608795880
creation_ttl        0s
display_name        root
entity_id           n/a
expire_time         <nil>
explicit_max_ttl    0s
id                  s.SxNruZ2GrTRBi3qgg29gTPRx
meta                <nil>
num_uses            0
orphan              true
path                auth/token/root
policies            [root]
ttl                 0s
type                service


---
### Rekeying Vault

The process for generating a new master key and applying Shamir's algorithm is called **"rekeying"**.  During the initialization, the encryption keys and unseal keys were generated.  This only happens **once** when the server is started against a new backend that has never been used with Vault before.

Under some circumstances, you may want to re-generate the master key and key shares.  For example:  
* Someone joints or leaves the organization  
* Security wants to hange the number of shares or threshold of shares  
* Compliance mandates the master key to be rotated at regular interval

<img src="images/vault-rekey-rotate.png" width="500">

##### 1. Initialize a rekeying operation

In [29]:
vault operator rekey -init -key-shares=3 -key-threshold=2  -format=json 

{
  "nonce": "5b4c17ce-70fd-5103-06d4-ba80563440a9",
  "started": true,
  "t": 2,
  "n": 3,
  "progress": 0,
  "required": 1,
  "pgp_fingerprints": null,
  "backup": false,
  "verification_required": false,
  "verification_nonce": ""
}


&nbsp;
##### 2. Each key holder must provide this nonce to rekey.  
This nonce is NOT a secret, so it is safe to distribute over insucre channels.   
Since there is only *one* unseal key, execute the following to rekey:

In [32]:
vault operator rekey -nonce="5b4c17ce-70fd-5103-06d4-ba80563440a9" "peXBqC0uG4Sgu0oH3HLDW6tB36247TfTuu382P0BJNs="


Key 1: nbj1hcD0z73B6ohMpjwlbiexKBYG9M40x5BoQHwFJq2q
Key 2: OCWENVorPALem9Z7cRJIQehOB9pBd47R9ypPqtfayXci
Key 3: gpJYqIl2jZEER7d4Rmkf/GGgugJY3PMWkEqYY6WH89my

Operation nonce: 5b4c17ce-70fd-5103-06d4-ba80563440a9

Vault rekeyed with 3 key shares and a key threshold of 2. Please securely
distribute the key shares printed above. When Vault is re-sealed, restarted,
or stopped, you must supply at least 2 of these keys to unseal it before it
can start servicing requests.


**The generated keys are the new unseal keys required to unseal your Vault server**

&nbsp;
### Rotating Encryption Key
The process for generating a new encryption key for Vault to encrypt data at rest is called **"rotating"**.  NIST recommends no more than 2^32 (~4 Billion) operations per key. 

**Note:** the **encryption key** is **never available** to operators and thus does NOT require a threshold of key holders.
To trigger a key rotation, execute the following:

In [33]:
vault operator rotate

Success! Rotated key

Key Term        2
Install Time    24 Dec 20 08:30 UTC


**The output shows the key version and installation time.  
This will add a new key to the keyring.  All new values written to the storage backend will be encrypted with this new key.  Old values can still be decrypted and are updated on-the-fly.**

---
### View Vault via a browser
Open a web browser and visit http://127.0.0.1:8200/ui/vault
Use **TOKEN** and **Sign in**

Only when you start the Vault server in development mode, you would see the key/value v2 secrets engine enabled at "secret/" path by default. However, when you run Vault in non-development mode, you would need to enable key/value secrets engine explicitly like other secrets engine.

&nbsp;

---
#### Thank you.
<img src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=100 align="left">